# 📊 End-to-End Sales Forecasting & Business Intelligence Dashboard

## 🚀 Project Summary

This project demonstrates how Machine Learning & Business Intelligence can be combined to build an executive sales forecasting dashboard.

The solution simulates a real-world analytics workflow:

**Raw Data → ETL → Forecast Model → Data Warehouse → Power BI Dashboard**

### 🎯 Objectives

* Transform raw retail sales data into analytics-ready datasets
* Build a time-series forecasting model
* Evaluate model performance using ML metrics
* Integrate predictions into a BI dashboard
* Create executive-level KPIs & interactive analytics

### 🏗️ Architecture Overview:

Raw Sales Data->
Python ETL Pipeline->
Time-Series Forecast Model->
Prediction Export (CSV)->
Power BI Star Schema->
Executive Dashboard

### 🤖 Machine Learning & Forecasting

Sales forecasting was performed using Facebook Prophet.

**-📏 Model Evaluation**

- MAE: 14,501

- RMSE: 19,156

- MAPE: 17.8%

The model captures trend & seasonality, making it suitable for business planning.

### ⚙️ Data Engineering & ETL

Python pipeline performs:

- Date parsing & feature engineering

- Monthly aggregation of sales

- Category-level forecasting

- Export of BI-ready datasets

 Libraries Used :

- Pandas

- NumPy

- Prophet

### 🧩 Data Modeling & Star Schema

A star schema was designed in Power BI.

**⭐ Fact Table**

fact_sales → historical transactions

**📅 Dimension Table**

calendar → continuous date table

**🔮 Prediction Table**

category_forecasts → ML predictions

This enables scalable & performant analytics.

### 📈 Business KPIs (DAX)

Executive metrics created in Power BI:

- Total Revenue

- Forecast Revenue

- Forecast Gap

- Month-over-Month Growth

- Year-over-Year Growth

These KPIs support strategic planning & performance monitoring.

### 📊 Dashboard Highlights
- Executive Overview
- Revenue KPI cards
- Forecast vs Actual trend
- Growth indicators
- Performance Insights
- Revenue by Region
- Revenue by Product Category
- Interactive Date Filtering

### 🛠️ Tech Stack
- Area	Tools
- Data Processing	Python & Pandas
- Machine Learning	Prophet
- BI & Visualization	Power BI
- Analytics	DAX

### 💼 Business Value

This solution demonstrates how predictive analytics can enable:

- Revenue planning & budgeting

- Demand forecasting

- Performance monitoring

- Data-driven decision making

### 📌 Conclusion

This project showcases an end-to-end analytics pipeline combining:

* Data Engineering + Machine Learning + Business Intelligence to deliver actionable insights for business users.

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/datasets/rohitsahoo/sales-forecasting/train.csv")
df.head()


In [ ]:
df.info()

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

df['year'] = df['Order Date'].dt.year
df['month'] = df['Order Date'].dt.month



In [ ]:
df[['Order Date','year','month']].head()


In [ ]:
# BI dataset (for Power BI visuals)
bi_sales = (
    df.groupby(['year','month','Region','Category'])['Sales']
    .sum()
    .reset_index()
)

bi_sales.head()


In [ ]:
#time-series dataset (for ML)
ts = (
    df.groupby('Order Date')['Sales']
    .sum()
    .reset_index()
)

ts = ts.resample('MS', on='Order Date').sum().reset_index()
ts.rename(columns={'Order Date':'ds','Sales':'y'}, inplace=True)

ts.head()


In [ ]:
!pip install prophet


In [ ]:

# Training of the data 
from prophet import Prophet

model = Prophet()
model.fit(ts)

future = model.make_future_dataframe(periods=3, freq='MS')
forecast = model.predict(future)

forecast[['ds','yhat']].tail()


In [ ]:
# Exporting the data into csv format
bi_sales.to_csv("/kaggle/working/bi_sales_table.csv", index=False)
forecast[['ds','yhat']].to_csv("/kaggle/working/sales_forecast.csv", index=False)


## Model Evaluation

This section has been carefully crafted to facilitate deeper analysis, allowing us to uncover valuable insights from the data. It is completed to provide further analysis and gain more insights into aspects such as category-wise breakdown, revenue, and order data.

In [ ]:
# last 6 months = test
train = ts[:-6]
test = ts[-6:]

train.shape, test.shape


In [ ]:
from prophet import Prophet

model = Prophet()
model.fit(train)


In [ ]:
future = model.make_future_dataframe(periods=6, freq='MS')
forecast = model.predict(future)

pred_test = forecast.tail(6)[['ds','yhat']]
pred_test


In [ ]:
eval_df = test.merge(pred_test, on='ds')
eval_df


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(eval_df['y'], eval_df['yhat'])
rmse = np.sqrt(mean_squared_error(eval_df['y'], eval_df['yhat']))

print("MAE:", mae)
print("RMSE:", rmse)


In [ ]:
mape = np.mean(np.abs((eval_df['y'] - eval_df['yhat']) / eval_df['y'])) * 100
print("MAPE:", mape, "%")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(eval_df['ds'], eval_df['y'], label='Actual')
plt.plot(eval_df['ds'], eval_df['yhat'], label='Predicted')
plt.legend()
plt.title("Forecast vs Actual Sales")
plt.show()


# Forecast per Category (Business-realistic modeling)

In [ ]:
categories = df['Category'].unique()
category_forecasts = []

from prophet import Prophet

for cat in categories:
    cat_df = df[df['Category'] == cat]
    
    ts_cat = (
        cat_df.groupby('Order Date')['Sales']
        .sum()
        .resample('MS')
        .sum()
        .reset_index()
    )
    
    ts_cat.rename(columns={'Order Date':'ds','Sales':'y'}, inplace=True)
    
    model = Prophet(yearly_seasonality=True)
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.fit(ts_cat)
    
    future = model.make_future_dataframe(periods=3, freq='MS')
    forecast_cat = model.predict(future)[['ds','yhat']]
    forecast_cat['Category'] = cat
    
    category_forecasts.append(forecast_cat)

category_forecasts = pd.concat(category_forecasts)
category_forecasts.head()


In [ ]:
# fact + dimension tables like a real BI system
# Dimensions table
dim_date = df[['Order Date']].drop_duplicates()
dim_date['Year'] = dim_date['Order Date'].dt.year
dim_date['Month'] = dim_date['Order Date'].dt.month
dim_date['MonthName'] = dim_date['Order Date'].dt.month_name()

dim_region = df[['Region']].drop_duplicates()
dim_category = df[['Category','Sub-Category']].drop_duplicates()

# Fact table
fact_sales = df[['Order Date','Region','Category','Sales']]


In [ ]:
# Exporting the files into CSV for power BI
dim_date.to_csv("/kaggle/working/dim_date.csv", index=False)
dim_region.to_csv("/kaggle/working/dim_region.csv", index=False)
dim_category.to_csv("/kaggle/working/dim_category.csv", index=False)
fact_sales.to_csv("/kaggle/working/fact_sales.csv", index=False)
category_forecasts.to_csv("/kaggle/working/category_forecasts.csv", index=False)
